In [1]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 23.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1Hydbg7KXOoK9Xf-DPFsoMGv6Whdzhh1F'})
download.GetContentFile('train_full.csv')
download = drive.CreateFile({'id': '1uo-5rbDJKUSBondu2NnaMC4ygu1TwuW0'})
download.GetContentFile('test_full.csv')

In [0]:
import pandas as pd
import numpy as np

label_name = 'insult'

train_full = pd.read_csv('train_full.csv')
test_full = pd.read_csv('test_full.csv')



In [5]:
#####INSULT

test1 = test_full[ test_full[label_name]==1 ]
test0 = test_full[ test_full[label_name]==0 ]

train1 = train_full[ train_full[label_name]==1 ]
train0 = train_full[ train_full[label_name]==0 ]

test0_new = test0.iloc[0:3000,:]
train0_new = train0.iloc[0:5000,:]


train_full = pd.concat([train1,train0_new],axis='rows',ignore_index=True).reset_index(drop=True)
test_full = pd.concat([test1,test0_new],axis='rows',ignore_index=True).reset_index(drop=True)

#shuffling concatenated dataframe:
train_full = train_full.sample(frac=1).reset_index(drop=True)
test_full = test_full.sample(frac=1).reset_index(drop=True)

print(train_full.shape,test_full.shape,list(train_full))

(9864, 9) (5426, 9) ['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'non-toxic', 'comment_text']


Create word vectors

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report

In [0]:
train = train_full["comment_text"]
test = test_full["comment_text"]

trainlab = train_full[label_name]
testlab = test_full[label_name]

In [8]:
# string cleaning- already done
# Download and load the punkt tokenizer for sentence splitting
import nltk.data
nltk.download("popular")   
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [0]:
# to split a review into parsed sentences returning a list of sentences (each sentence is list of words)
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    review = str(review)
    raw_sentences = tokenizer.tokenize(review.strip()) # split para into sentences
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            #sentences.append( review_to_wordlist(raw_sentence,remove_stopwords) )
            sentences.append(raw_sentence) #bcz already cleaned h
    return sentences

In [10]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for r in train:
  sentences += review_to_sentences(r, tokenizer)

print ("Parsing sentences from testing set")
for r in test:
    sentences += review_to_sentences(r, tokenizer)
    
print(sentences[0:5])

Parsing sentences from training set
Parsing sentences from testing set
['utc official wikipedia doxa rs homosexuality calomny may', 'sorry must reading something else think needed posting disciplines articles diving ex cycling also remove since one article also team sport rosters templates', 'go fuck traitor thank', 'think kill jews homosexuals handicaped people said adolf hitler', 'warning nothing fc k u fc k get warning nothing fc k u fc k get warning nothing fc k u fc k get warning nothing']


In [0]:
#training and saving model 
import logging # for nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [12]:
# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
div_factor = 1000    # division factor for clusters
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

2019-04-19 09:28:08,963 : WARNING : paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
2019-04-19 09:28:09,097 : INFO : 'pattern' package not found; tag filters are not available for English
2019-04-19 09:28:09,109 : INFO : collecting all words and their counts
2019-04-19 09:28:09,110 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2019-04-19 09:28:09,111 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2019-04-19 09:28:09,367 : INFO : PROGRESS: at sentence #10000, processed 2082751 words, keeping 27 word types
2019-04-19 09:28:09,497 : INFO : collected 27 word types from a corpus of 3150444 raw words and 15290 sentences
2019-04-19 09:28:09,498 : INFO : Loading a fresh vocabulary
2019-04-19 09:28:09,499 : INFO : effective_min_count=40 retains 27 unique words (100% of original 27, drops 0)
2019-04-19 09:28:09,500 : INFO : effective_min_count=40 leaves 3150444 word corpus (100% of original 3150444, drops 0)
2019-04-19 09:28:09,501 : INFO : deleting the raw counts dictionary of 27 items
2019-04-19 09:28:09,504 : INFO : sample=0.001 downsamples 23 most-common words
2019-04-19 09:28:09,506 : INFO : downsampling leaves estimated 543933 word corpus (17.3% of prior 3150444)
2019-04-19 09:28:09,507 : INFO : estimated required memory for 27 words and 100 dimensions: 35100 bytes
2019-04-19 09:28:09,509 : INFO : resetting layer weights
2019-04-19 09:28:09,511 : INFO : training model with 4 worker

Converting review to word vectors

In [0]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
       print(reviewFeatureVecs)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [14]:
trainDataVecs = getAvgFeatureVecs( train, model, num_features )
testDataVecs = getAvgFeatureVecs( test, model, num_features )


Output hidden; open in https://colab.research.google.com to view.

Clustering these vectors

In [15]:
type(trainDataVecs)
print(trainDataVecs.shape)
tot = len(trainDataVecs)+len(testDataVecs)
print(tot)

arr = np.zeros((tot, num_features))

for col in range(0,num_features):
  for row in range(0, len(trainDataVecs)):
    arr[row] = np.array(trainDataVecs[row])
  
  
print(arr.shape)
l1 = len(trainDataVecs)
l2 = len(testDataVecs)
  
  
for col in range(0,num_features):
  for row in range(0, len(testDataVecs)):
    arr[l1 + row] = np.array(testDataVecs[row])


#word_vectors = np.concatenate(trainDataVecs,testDataVecs)
#print(word_vectors.shape)


(9864, 100)
15290
(15290, 100)


In [16]:
word_vectors = np.matrix(arr)
print(word_vectors.shape)

(15290, 100)


In [17]:
#word_vectors = pd.concat([trainDataVecs,testDataVecs])
#num_clusters = int(len(word_vectors) / div_factor)
#from sklearn.cluster import DBSCAN


num_clusters = 50

# Initalize a k-means object and use it to extract centroids
import time

start = time.time() # Start time


clusterer = KMeans( n_clusters = num_clusters, max_iter=500,  n_init=20)

idx = clusterer.fit_predict( word_vectors )


# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")
print (idx)

Time taken for K Means clustering:  34.752663373947144 seconds.
[17 47 35 ...  8 12 17]


BAG OF CENTROIDS


In [0]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [19]:
# For the first 10 clusters
for cluster in range(0,50):
    #
    # Print the cluster number  
    print ("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print (words)


Cluster 0
[]

Cluster 1
['j']

Cluster 2
[]

Cluster 3
[]

Cluster 4
[]

Cluster 5
['b']

Cluster 6
[]

Cluster 7
['m']

Cluster 8
['q']

Cluster 9
[]

Cluster 10
[]

Cluster 11
[]

Cluster 12
['w']

Cluster 13
[]

Cluster 14
[]

Cluster 15
[]

Cluster 16
[]

Cluster 17
[' ']

Cluster 18
[]

Cluster 19
[]

Cluster 20
[]

Cluster 21
[]

Cluster 22
['u', 'k', 'f']

Cluster 23
['l']

Cluster 24
[]

Cluster 25
['a', 'h']

Cluster 26
[]

Cluster 27
['d', 'z']

Cluster 28
[]

Cluster 29
[]

Cluster 30
[]

Cluster 31
['o', 'r']

Cluster 32
['t', 'p']

Cluster 33
[]

Cluster 34
[]

Cluster 35
['i']

Cluster 36
['n', 'v']

Cluster 37
[]

Cluster 38
['c']

Cluster 39
[]

Cluster 40
[]

Cluster 41
[]

Cluster 42
[]

Cluster 43
['y']

Cluster 44
[]

Cluster 45
[]

Cluster 46
[]

Cluster 47
['e', 'g', 'x']

Cluster 48
[]

Cluster 49
['s']


In [0]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = num_clusters
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    #print(len(bag_of_centroids))
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    #print(len(bag_of_centroids))
    return bag_of_centroids

In [0]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train.size, num_clusters), dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in train:
    train_centroids[counter][:] =  create_bag_of_centroids( review, word_centroid_map ) 
    counter = counter + 1

# Repeat for test reviews 
test_centroids = np.zeros(( test.size, num_clusters), dtype="float32" )

counter = 0
for review in test:
    test_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

SVC poly

In [0]:
from sklearn.svm import SVC
# Fit a random forest and extract predictions 
forest = SVC(kernel='poly')

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train_full[label_name])
result = forest.predict(test_centroids)

# Write the test results 


Fitting a random forest to labeled training data...


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [0]:
from sklearn.metrics import classification_report
#test data
print(classification_report(testlab, result))

Random forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
# Fit a random forest and extract predictions 
forest = RandomForestClassifier()
# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train_full[label_name])
result = forest.predict(test_centroids)

# Write the test results 


Fitting a random forest to labeled training data...


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [25]:
from sklearn.metrics import classification_report
#test data
print(classification_report(testlab, result))

              precision    recall  f1-score   support

           0       0.68      0.74      0.71      3000
           1       0.64      0.57      0.60      2426

   micro avg       0.66      0.66      0.66      5426
   macro avg       0.66      0.66      0.66      5426
weighted avg       0.66      0.66      0.66      5426

